# Homework 6: 1, 2, 9 

# 1.

The best subset selection method will have the smallest training RSS among the three methods for a given number of predictors 
𝑘
k. This is because best subset selection evaluates all possible combinations of 
𝑘
k predictors and chooses the one that minimizes the training error. In contrast, forward and backward stepwise selection only explore a limited subset of models, so they may not find the model with the absolute lowest training RSS.